# EEG Signal Analysis Template

This notebook demonstrates basic EEG signal processing and analysis techniques for the EEG Lab platform.

## Objectives
- Load and visualize EEG data
- Perform basic signal preprocessing
- Extract features for machine learning
- Implement a simple classification model

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.stats import zscore
import mne

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Deep learning libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Set plotting parameters
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"MNE version: {mne.__version__}")
print(f"PyTorch version: {torch.__version__}")

## 1. Data Loading and Exploration

In this section, we'll load EEG data and explore its basic properties.

In [ ]:
# Example: Create synthetic EEG data for demonstration
# In practice, you would load real EEG data using MNE or other tools

def create_synthetic_eeg(n_channels=32, n_samples=1000, sampling_freq=256):
    """
    Create synthetic EEG data for demonstration purposes.
    
    Parameters:
    - n_channels: Number of EEG channels
    - n_samples: Number of time samples
    - sampling_freq: Sampling frequency in Hz
    
    Returns:
    - data: EEG data array (channels x samples)
    - time: Time vector
    """
    np.random.seed(42)
    time = np.linspace(0, n_samples/sampling_freq, n_samples)
    
    # Create synthetic EEG with multiple frequency components
    data = np.zeros((n_channels, n_samples))
    
    for ch in range(n_channels):
        # Alpha rhythm (8-12 Hz)
        alpha = 2 * np.sin(2 * np.pi * 10 * time + np.random.random())
        
        # Beta rhythm (13-30 Hz)
        beta = 1 * np.sin(2 * np.pi * 20 * time + np.random.random())
        
        # Gamma rhythm (30-100 Hz)
        gamma = 0.5 * np.sin(2 * np.pi * 40 * time + np.random.random())
        
        # Add noise
        noise = np.random.normal(0, 0.5, n_samples)
        
        data[ch] = alpha + beta + gamma + noise
    
    return data, time

# Generate synthetic data
eeg_data, time_vector = create_synthetic_eeg(n_channels=19, n_samples=2560, sampling_freq=256)

print(f"EEG data shape: {eeg_data.shape}")
print(f"Number of channels: {eeg_data.shape[0]}")
print(f"Number of samples: {eeg_data.shape[1]}")
print(f"Duration: {eeg_data.shape[1]/256:.2f} seconds")

In [ ]:
# Visualize EEG channels
fig, axes = plt.subplots(4, 1, figsize=(15, 10))
fig.suptitle('EEG Signal Visualization', fontsize=16, fontweight='bold')

# Plot first 4 channels
channel_names = ['Fp1', 'Fp2', 'F3', 'F4']  # Example channel names

for i, ax in enumerate(axes):
    ax.plot(time_vector, eeg_data[i], linewidth=0.8)
    ax.set_title(f'Channel {channel_names[i]}', fontweight='bold')
    ax.set_ylabel('Amplitude (μV)')
    ax.grid(True, alpha=0.3)
    
axes[-1].set_xlabel('Time (seconds)')
plt.tight_layout()
plt.show()

# Display basic statistics
print("\nBasic Statistics:")
print(f"Mean amplitude: {np.mean(eeg_data):.3f} μV")
print(f"Standard deviation: {np.std(eeg_data):.3f} μV")
print(f"Min amplitude: {np.min(eeg_data):.3f} μV")
print(f"Max amplitude: {np.max(eeg_data):.3f} μV")

## 2. Signal Preprocessing

Apply common preprocessing steps to clean the EEG signal.

In [ ]:
def preprocess_eeg(data, sampling_freq=256, lowpass=50, highpass=0.5):
    """
    Apply basic preprocessing to EEG data.
    
    Parameters:
    - data: EEG data (channels x samples)
    - sampling_freq: Sampling frequency
    - lowpass: Low-pass filter cutoff frequency
    - highpass: High-pass filter cutoff frequency
    
    Returns:
    - processed_data: Preprocessed EEG data
    """
    processed_data = data.copy()
    
    # 1. Remove DC offset (mean centering)
    processed_data = processed_data - np.mean(processed_data, axis=1, keepdims=True)
    
    # 2. Apply bandpass filter
    nyquist = sampling_freq / 2
    low = highpass / nyquist
    high = lowpass / nyquist
    
    b, a = signal.butter(4, [low, high], btype='band')
    
    for ch in range(processed_data.shape[0]):
        processed_data[ch] = signal.filtfilt(b, a, processed_data[ch])
    
    # 3. Z-score normalization
    processed_data = zscore(processed_data, axis=1)
    
    return processed_data

# Apply preprocessing
preprocessed_eeg = preprocess_eeg(eeg_data)

# Compare original and preprocessed signals
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
fig.suptitle('Original vs Preprocessed EEG Signals', fontsize=16, fontweight='bold')

# Original signals
axes[0, 0].plot(time_vector, eeg_data[0])
axes[0, 0].set_title('Original Signal - Channel Fp1')
axes[0, 0].set_ylabel('Amplitude (μV)')

axes[0, 1].plot(time_vector, eeg_data[1])
axes[0, 1].set_title('Original Signal - Channel Fp2')
axes[0, 1].set_ylabel('Amplitude (μV)')

# Preprocessed signals
axes[1, 0].plot(time_vector, preprocessed_eeg[0])
axes[1, 0].set_title('Preprocessed Signal - Channel Fp1')
axes[1, 0].set_ylabel('Normalized Amplitude')
axes[1, 0].set_xlabel('Time (seconds)')

axes[1, 1].plot(time_vector, preprocessed_eeg[1])
axes[1, 1].set_title('Preprocessed Signal - Channel Fp2')
axes[1, 1].set_ylabel('Normalized Amplitude')
axes[1, 1].set_xlabel('Time (seconds)')

plt.tight_layout()
plt.show()

## 3. Feature Extraction

Extract relevant features from the EEG signal for machine learning.

In [ ]:
def extract_eeg_features(data, sampling_freq=256):
    """
    Extract features from EEG data.
    
    Parameters:
    - data: EEG data (channels x samples)
    - sampling_freq: Sampling frequency
    
    Returns:
    - features: Dictionary of extracted features
    """
    features = {}
    n_channels, n_samples = data.shape
    
    # Time domain features
    features['mean'] = np.mean(data, axis=1)
    features['std'] = np.std(data, axis=1)
    features['var'] = np.var(data, axis=1)
    features['skewness'] = np.array([pd.Series(data[ch]).skew() for ch in range(n_channels)])
    features['kurtosis'] = np.array([pd.Series(data[ch]).kurtosis() for ch in range(n_channels)])
    
    # Frequency domain features
    freqs, psd = signal.welch(data, sampling_freq, nperseg=256)
    
    # Define frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 50)
    }
    
    for band_name, (low_freq, high_freq) in bands.items():
        band_mask = (freqs >= low_freq) & (freqs <= high_freq)
        band_power = np.mean(psd[:, band_mask], axis=1)
        features[f'{band_name}_power'] = band_power
    
    # Relative power features
    total_power = np.sum(psd, axis=1)
    for band_name in bands.keys():
        features[f'{band_name}_relative'] = features[f'{band_name}_power'] / total_power
    
    return features

# Extract features
eeg_features = extract_eeg_features(preprocessed_eeg)

print("Extracted Features:")
for feature_name, feature_values in eeg_features.items():
    print(f"{feature_name}: shape {feature_values.shape}")

# Visualize frequency band powers
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
band_powers = np.array([eeg_features[f'{band}_power'] for band in band_names])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Average power across channels for each band
avg_powers = np.mean(band_powers, axis=1)
ax1.bar(band_names, avg_powers)
ax1.set_title('Average Power Spectral Density by Frequency Band')
ax1.set_ylabel('Power (μV²/Hz)')
ax1.set_xlabel('Frequency Band')

# Heatmap of band powers across channels
im = ax2.imshow(band_powers, aspect='auto', cmap='viridis')
ax2.set_title('Power Spectral Density Heatmap')
ax2.set_xlabel('EEG Channels')
ax2.set_ylabel('Frequency Bands')
ax2.set_yticks(range(len(band_names)))
ax2.set_yticklabels(band_names)
plt.colorbar(im, ax=ax2, label='Power (μV²/Hz)')

plt.tight_layout()
plt.show()

## 4. Machine Learning Model

Create a simple classification model using the extracted features.

In [ ]:
# Create synthetic dataset for classification
def create_classification_dataset(n_samples=1000, n_channels=19):
    """
    Create a synthetic dataset for EEG classification.
    
    Returns:
    - X: Feature matrix
    - y: Labels (0: normal, 1: abnormal)
    """
    np.random.seed(42)
    
    X = []
    y = []
    
    for i in range(n_samples):
        # Generate EEG sample
        if i < n_samples // 2:
            # Normal EEG (class 0)
            data, _ = create_synthetic_eeg(n_channels=n_channels, n_samples=1280)
            label = 0
        else:
            # Abnormal EEG (class 1) - simulate seizure-like activity
            data, time = create_synthetic_eeg(n_channels=n_channels, n_samples=1280)
            # Add high-frequency artifacts
            seizure_signal = 5 * np.sin(2 * np.pi * 15 * time)
            data = data + seizure_signal
            label = 1
        
        # Preprocess and extract features
        processed_data = preprocess_eeg(data)
        features = extract_eeg_features(processed_data)
        
        # Flatten features into a single vector
        feature_vector = np.concatenate([features[key] for key in features.keys()])
        
        X.append(feature_vector)
        y.append(label)
    
    return np.array(X), np.array(y)

print("Creating synthetic dataset...")
X, y = create_classification_dataset(n_samples=200)

print(f"Dataset shape: {X.shape}")
print(f"Number of features: {X.shape[1]}")
print(f"Class distribution: {np.bincount(y)}")

In [ ]:
# Train a Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)
y_prob = rf_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Abnormal']))

# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Abnormal'], 
            yticklabels=['Normal', 'Abnormal'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Feature importance
feature_names = []
for key in eeg_features.keys():
    for ch in range(19):  # 19 channels
        feature_names.append(f"{key}_ch{ch}")

importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1][:20]  # Top 20 features

plt.figure(figsize=(12, 6))
plt.title('Top 20 Feature Importances')
plt.bar(range(20), importances[indices])
plt.xticks(range(20), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 5. Deep Learning Model (PyTorch)

Implement a simple neural network for EEG classification.

In [ ]:
class EEGNet(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_classes=2):
        super(EEGNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(hidden_size // 2, num_classes)
        
    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)

# Initialize model
input_size = X_train_scaled.shape[1]
model = EEGNet(input_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
train_losses = []
train_accuracies = []

print("Training Neural Network...")
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # Calculate training accuracy
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_train_tensor).float().mean()
    
    train_losses.append(loss.item())
    train_accuracies.append(accuracy.item())
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}')

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs.data, 1)
    test_accuracy = (test_predicted == y_test_tensor).float().mean()
    
print(f'\nTest Accuracy: {test_accuracy.item():.4f}')

# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses)
ax1.set_title('Training Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True)

ax2.plot(train_accuracies)
ax2.set_title('Training Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.show()

## 6. Model Comparison and Results

Compare the performance of different models and summarize results.

In [ ]:
# Compare model performance
rf_accuracy = accuracy_score(y_test, y_pred)
nn_accuracy = test_accuracy.item()

models = ['Random Forest', 'Neural Network']
accuracies = [rf_accuracy, nn_accuracy]

plt.figure(figsize=(8, 6))
bars = plt.bar(models, accuracies, color=['skyblue', 'lightcoral'])
plt.title('Model Performance Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy')
plt.ylim(0, 1)

# Add accuracy values on bars
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3)
plt.show()

# Summary
print("\n" + "="*50)
print("EEG ANALYSIS SUMMARY")
print("="*50)
print(f"Dataset size: {len(X)} samples")
print(f"Number of features extracted: {X.shape[1]}")
print(f"Number of EEG channels: 19")
print(f"\nModel Performance:")
print(f"Random Forest Accuracy: {rf_accuracy:.3f}")
print(f"Neural Network Accuracy: {nn_accuracy:.3f}")
print(f"\nBest performing model: {'Neural Network' if nn_accuracy > rf_accuracy else 'Random Forest'}")
print("="*50)

## 7. Save Results and Model

Save the trained models and results for later use.

In [ ]:
import pickle
import os

# Create results directory if it doesn't exist
results_dir = '../results'
models_dir = '../models'

os.makedirs(results_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# Save Random Forest model
with open(f'{models_dir}/random_forest_model.pkl', 'wb') as f:
    pickle.dump({'model': rf_model, 'scaler': scaler}, f)

# Save PyTorch model
torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': input_size,
    'model_class': 'EEGNet'
}, f'{models_dir}/eeg_neural_network.pth')

# Save results
results = {
    'dataset_info': {
        'n_samples': len(X),
        'n_features': X.shape[1],
        'n_channels': 19
    },
    'model_performance': {
        'random_forest': rf_accuracy,
        'neural_network': nn_accuracy
    },
    'feature_importance': dict(zip([feature_names[i] for i in indices[:10]], 
                                 importances[indices[:10]]))
}

with open(f'{results_dir}/analysis_results.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Models and results saved successfully!")
print(f"Random Forest model: {models_dir}/random_forest_model.pkl")
print(f"Neural Network model: {models_dir}/eeg_neural_network.pth")
print(f"Analysis results: {results_dir}/analysis_results.pkl")

## Conclusion

This notebook demonstrated a complete EEG analysis pipeline including:

1. **Data Loading**: Created synthetic EEG data for demonstration
2. **Preprocessing**: Applied filtering and normalization
3. **Feature Extraction**: Extracted time and frequency domain features
4. **Machine Learning**: Trained both traditional ML and deep learning models
5. **Evaluation**: Compared model performance
6. **Saving**: Stored models and results for future use

### Next Steps
- Load real EEG datasets
- Implement more sophisticated preprocessing techniques
- Explore advanced deep learning architectures (CNN, RNN, Transformers)
- Perform cross-validation and hyperparameter tuning
- Implement real-time EEG analysis capabilities

### Resources
- [MNE-Python Documentation](https://mne.tools/)
- [EEG Analysis Tutorials](https://mne.tools/stable/auto_tutorials/index.html)
- [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)